In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score

# 미리 전처리 해놓은 201407 val aws csv 데이터 불러와서 학습 데이터와 같은 방법으로 nan 값 처리
val_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/201407.csv")
# val_csv

regionlist_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/region_filled.csv')
regionlist = regionlist_csv['지점'][0:628].tolist()
for r in regionlist:
    y = val_csv.loc[val_csv['지점번호'] == r]
    globals()[f'aws_{int(r)}'] = y # aws_지점번호 인 csv 생성 및 변수명 할당
    globals()[f'aws_{int(r)}'] =  globals()[f'aws_{int(r)}'].interpolate()

aws = pd.DataFrame()
for r in regionlist:
    aws = pd.concat([aws, globals()[f'aws_{int(r)}']])
aws = aws.fillna(0)
aws

,Unnamed: 0,지점번호,시각,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,합성 풍향,최대 순간 풍속,...,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고,위도,경도,해발고도,권역,weather_label
0,0,90,2014-07-01 01:00:00,20.3,20.5,20.0,0.7,0.6,285.6,1.7,...,1006.9,1009.0,20.1,0.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
1,1,90,2014-07-01 02:00:00,20.1,20.3,19.9,0.8,0.7,310.0,1.7,...,1007.2,1009.3,19.9,0.0,7620.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
2,2,90,2014-07-01 03:00:00,19.9,20.0,19.7,1.2,1.1,306.1,2.5,...,1007.4,1009.5,19.6,0.0,45.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
3,3,90,2014-07-01 04:00:00,19.6,19.7,19.4,0.4,0.2,353.8,1.6,...,1007.6,1009.7,19.5,0.0,45.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
4,4,90,2014-07-01 05:00:00,19.5,19.7,19.3,0.5,0.4,325.3,1.2,...,1008.2,1010.3,19.2,0.0,45.0,38.2509,128.5647,17.53,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467227,467227,989,2014-07-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '1', '0', '0', '0', '0', '0', '0', ..."
467228,467228,989,2014-07-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '1', '0', '0', '0', '0', '0', '0', ..."
467229,467229,989,2014-07-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '1', '1', '0', '0', '0', '0', '0', ..."
467230,467230,989,2014-07-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.2579,126.3299,90.00,제주도,"['0', '0', '1', '1', '0', '0', '0', '0', '0', ..."


In [2]:
# 같은 방법으로 normalization도 진행
csv_features = ['평균 기온', '최고 기온', '최저 기온', '평균 풍속', '합성 풍속', '합성 풍향', '최대 순간 풍속', '최대 순간 풍속 풍향', 
                '평균 전운량', '합계 강수량', '평균 현지기압', '평균 해면기압', '이슬점온도', '10분 평균 시정', '운고', '위도', '경도',
               '해발고도']

temp_csv = aws[csv_features]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

aws_norm = pd.DataFrame(scaler.fit_transform(temp_csv), columns=temp_csv.columns)
val_data = aws.drop(csv_features, axis=1)
val_data = pd.concat([val_data, aws_norm], axis=1)
val_data

,Unnamed: 0,지점번호,시각,권역,weather_label,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,...,평균 전운량,합계 강수량,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고,위도,경도,해발고도
0,0,90,2014-07-01 01:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.523196,0.513784,0.542005,0.024055,0.020690,...,0.530186,0.0,0.991922,0.992817,0.688356,0.0,0.813321,0.992434,0.965860,0.009870
1,1,90,2014-07-01 02:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.518041,0.508772,0.539295,0.027491,0.024138,...,0.530186,0.0,0.992218,0.993112,0.681507,0.0,0.813321,0.992434,0.965860,0.009870
2,2,90,2014-07-01 03:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.512887,0.501253,0.533875,0.041237,0.037931,...,0.530186,0.0,0.992415,0.993309,0.671233,0.0,0.004803,0.992434,0.965860,0.009870
3,3,90,2014-07-01 04:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.505155,0.493734,0.525745,0.013746,0.006897,...,0.530186,0.0,0.992612,0.993506,0.667808,0.0,0.004803,0.992434,0.965860,0.009870
4,4,90,2014-07-01 05:00:00,강원도,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...",0.502577,0.493734,0.523035,0.017182,0.013793,...,0.530186,0.0,0.993203,0.994096,0.657534,0.0,0.004803,0.992434,0.965860,0.009870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467227,467227,989,2014-07-31 20:00:00,제주도,"['0', '0', '1', '0', '0', '0', '0', '0', '0', ...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.862889,0.942775,0.053406
467228,467228,989,2014-07-31 21:00:00,제주도,"['0', '0', '1', '0', '0', '0', '0', '0', '0', ...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.862889,0.942775,0.053406
467229,467229,989,2014-07-31 22:00:00,제주도,"['0', '0', '1', '1', '0', '0', '0', '0', '0', ...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.862889,0.942775,0.053406
467230,467230,989,2014-07-31 23:00:00,제주도,"['0', '0', '1', '1', '0', '0', '0', '0', '0', ...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.862889,0.942775,0.053406


In [4]:
from numpy import array
import ast
 
def split_sequences(sequences_x, sequence_y,  n_steps):
    X, y = list(), list()
    for i in range(len(sequence_y)):
        end_ix = i + n_steps
        if end_ix > len(sequence_y)-n_steps:
#         if end_ix > len(sequence_y):
            break
        seq_x = sequences_x[i:end_ix, :]
        seq_y = sequence_y[end_ix:end_ix+n_steps]
        seq_y_new = []
        for j in range(len(seq_y)):
            seq_y_new.append( list(map(int, ast.literal_eval(seq_y[j]))) )
#         seq_y = ast.literal_eval(seq_y)
#         seq_y = list(map(int, seq_y))
        X.append(seq_x)
#         y.append(seq_y)
        y.append(seq_y_new)

    return array(X),array(y)

In [5]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 
        self.n_layers = 1 
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
#         self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 13)
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 156)
#         self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden).cuda()
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden).cuda()
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [6]:
n_features = 18 
n_timesteps = 12 
device = torch.device("cuda:0")

save_path = "/raid/workspace/honghee/data_weather/train_x/aws/best_model_12.pt"

# convert dataset into input/output
X, y = split_sequences(training_x, training_y, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net = MV_LSTM(n_features,n_timesteps)
mv_net.load_state_dict(torch.load(save_path, map_location=device))


############# 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(467209, 12, 18) (467209, 12, 13)


<All keys matched successfully>

In [ ]:
# calculate MSE value

criterion = torch.nn.MSELoss() 

batch_size = 1
mv_net.to(device)
mv_net.eval()
with torch.no_grad():
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    

        x_batch = torch.tensor(inpt,dtype=torch.float32).cuda()    
        y_batch = torch.tensor(target,dtype=torch.float32).cuda()  

        mv_net.init_hidden(x_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_net(x_batch) 
        output = torch.reshape(output, (1, 12, 13))
    #     loss = criterion(output.view(-1), y_batch)
        loss = criterion(output, y_batch)
        
        print(loss)
        


In [8]:
from sklearn.metrics import f1_score
def accuracy_function(real, pred):    
    score = f1_score(real, pred, average='weighted')
    return score


In [9]:
# test and calculate f1-score
# calculate inference time

from sklearn import metrics
import time

batch_size = 1
mv_net.to(device)
mv_net.eval()
pred = []
label = []
with torch.no_grad():
    for b in range(0,len(X)-12,batch_size):
        start = time.time()
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    

        x_batch = torch.tensor(inpt,dtype=torch.float32).cuda()    
        y_batch = torch.tensor(target,dtype=torch.float32).cuda()  

        mv_net.init_hidden(x_batch.size(0))
        output = mv_net(x_batch) 
        output = torch.reshape(output, (1, 12, 13)).cpu().detach().numpy()
        if b == 0:
            t = time.time()-start
        output = output[:,:,2][0]
#         output = [0 if (x < 0.15841821 ) else 1 for x in output]
        output = [0 if (x < 0.15941821 ) else 1 for x in output] 
        pred.append(output)
        y_batch = y_batch[:,:,2][0].cpu().detach().numpy()
        y_batch = [0 if (x < 1 ) else 1 for x in y_batch] 
        label.append(y_batch)
    label = np.ravel(label)
    pred = np.ravel(pred)
    f1_score = accuracy_function(label, pred)
    print(f1_score)
#     med = np.median(pred, axis=0) 
#     print(np.median(med)) # 0.15841821

0.7478702360171589
